In [4]:
!pip install transformers datasets accelerate torch
!pip install arabert

/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [5]:
# Import necessary libraries
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import torch

In [6]:
dataset=pd.read_csv('/kaggle/input/aaaaaaaaaaaaaa/all_data_meta.csv',sep=None, engine='python', on_bad_lines='skip')

In [7]:
dataset

,answer_id,question_id,text,answer_start,answer_end,answer_category,question,file_name,open-domain,long,Unnamed: 10,Unnamed: 11
0,649960,649960,المعلومات المخزنة أو وسيلة التخزين.\nأي أن الب...,251,418,NaN,اذكر وسيله التخزين ؟,1338943.0,1,1.0,1 long 0 short,3167
1,649961,649961,يتكون البايت عادة من 8 بت، ولذلك فأن البايت ي...,417,536,NaN,مما يتكون البايت ؟,1338943.0,1,1.0,1 open 0 no internet,202
2,649962,649962,لاحظ أن الأسماء كيلوبايت وميجابايت ...الخ، يمك...,748,874,NaN,ما هي مضاعفات البايت ؟,1338943.0,1,1.0,NaN,NaN
3,649963,649963,وبتمديد النمط، نستطيع الحصول على وحدتين إضافيت...,882,1015,NaN,ما هو الاستخدام البايت ؟,1338943.0,1,1.0,NaN,NaN
4,649959,649959,البايت أو الثُّمَانِيَّة هي وحدة معلومات رقمية...,0,233,NaN,ما هو البايت ف الحاسوب ؟,1338943.0,1,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
60646,1191158,1191158,ريو دي جانيرو في البرازيل,70,95,NaN,أين ولد واشنطن ؟,1717957.0,واشنطن هو لاعب كرة قدم برازيلي في مركز الهجوم،...,1.0,0,None
60647,1191159,1191159,بوتافوغو ريغاتاس.,104,121,NaN,ما هو النادي الذي لعب معه واشنطن ؟,1717957.0,واشنطن هو لاعب كرة قدم برازيلي في مركز الهجوم،...,1.0,0,None
60648,1196592,1196592,تعود إلى عصور ما قبل التاريخ ونقوش صخرية تعود...,111,216,NaN,متى تعود النقوش الصخرية في منطقة الجلفة إلى؟\n,1717629.0,الفن الصخري بمنطقة الجلفة تتكون النقوش الصخرية...,1.0,1,None
60649,1196593,1196593,في الشرق في منطقة قسنطينة (الجزائر),350,386,NaN,ما هي المناطق الأخرى في الجزائر التي وُجِدت في...,1717629.0,الفن الصخري بمنطقة الجلفة تتكون النقوش الصخرية...,0.0,0,None


In [8]:
pip install torchmetrics


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install datasets evaluate transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# 1. Fine-tune the target model (see table below) on your training dataset and assess the performance on the test dataset

In [10]:
import os
import pandas as pd
from transformers import (
    GPT2Tokenizer,
    GPT2LMHeadModel,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline
)
from datasets import load_dataset

# Disable WandB (optional)
os.environ["WANDB_DISABLED"] = "true"

# Load metadata
metadata = pd.read_csv("/kaggle/input/aaaaaaaaaaaaaa/all_data_meta.csv", on_bad_lines='skip')

# Combine question and text columns
metadata['text'] = metadata['question'].fillna('') + " " + metadata['text'].fillna('')

# Save prepared text data
text_file_path = "finetune_data.txt"
with open(text_file_path, "w", encoding="utf-8") as f:
    for line in metadata['text']:
        f.write(line.strip() + "\n")

# Load AraGPT2 model and tokenizer
model_name = "aubmindlab/aragpt2-base"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

# Add special tokens (if necessary)
tokenizer.pad_token = tokenizer.eos_token

# Load the dataset
dataset = load_dataset('text', data_files={'train': text_file_path})

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

# Split the dataset into training and evaluation
train_test_split = tokenized_datasets["train"].train_test_split(test_size=0.1)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

# Define a data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # No masked language modeling
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="aragpt2-finetuning",  # Custom run name
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=8,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    evaluation_strategy="epoch",
    warmup_steps=500,
    weight_decay=0.01,
    fp16=True,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,  # Use data collator
)

# Train the model
trainer.train()




vocab.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/108094 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,4.367900,3.946282


TrainOutput(global_step=12161, training_loss=4.696073472034224, metrics={'train_runtime': 2479.4749, 'train_samples_per_second': 39.236, 'train_steps_per_second': 4.905, 'total_flos': 6354883510272000.0, 'train_loss': 4.696073472034224, 'epoch': 1.0})

test model

In [33]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, pipeline

# Charger le tokenizer et le modèle
model_path = "./fine_tuned_aragpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

# Ajouter des tokens spéciaux si nécessaire
tokenizer.pad_token = tokenizer.eos_token


In [34]:
# Préparer une entrée
input_text = "ما هي عاصمة المغرب؟"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Générer une sortie
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    pad_token_id=tokenizer.eos_token_id
)

# Décoder la sortie
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)


Generated Text: ما هي عاصمة المغرب؟ مدينة مراكش في المغرب. تقع في شمال غرب المغرب على بعد حوالي 70 كم جنوب غرب مدينة الدار البيضاء، عاصمة المملكة المغربية. يقع مقر شركة المغرب للاتصالات السلكية واللاسلكية في مدينة طنجة؛ في جنوب شرق المغرب في منطقة


In [35]:
# Préparer une entrée
input_text = "ما هو الاستخدام البايت ؟"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Générer une sortie
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    pad_token_id=tokenizer.eos_token_id
)

# Décoder la sortie
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)



Generated Text: ما هو الاستخدام البايت ؟  يستخدم في علاج حالات فرط ضغط الدم في المرضى الذين يعانون من نقص ضغط دم حاد أو نقص في الدم أو فقدان وظائف القلب. يمكن أن يؤدي نقص الدم إلى حدوث اضطراب في القلب أو حدوث ضمور في العضلات أو ضمور العضلات


In [36]:
# Préparer une entrée
input_text = "ين ولد واشنطن ؟"
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# Générer une sortie
output = model.generate(
    input_ids,
    max_length=50,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    pad_token_id=tokenizer.eos_token_id
)

# Décoder la sortie
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Text:", generated_text)





Generated Text: ين ولد واشنطن ؟  في الولايات المتحدة. في المملكة المتحدة في عام 1994. ولد في مدينة نيويورك في نيويورك. كان والده في سن الثامنة عشر من عمره. بعد وفاة والده، انتقل إلى الولايات الأمريكية. التحق بمدرسة الفنون الجميلة في جامعة نيويورك�


load model

In [21]:
# Define the path where you want to save the model and tokenizer
save_path = "./fine_tuned_aragpt2"

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model and tokenizer saved to {save_path}")


Model and tokenizer saved to ./fine_tuned_aragpt2


In [22]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the fine-tuned model and tokenizer
model = GPT2LMHeadModel.from_pretrained("./fine_tuned_aragpt2")
tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_aragpt2")


# 2. Implement a multi-head attention class based on differential attention 

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(DifferentialAttention, self).__init__()
        assert d_model % (2 * num_heads) == 0, "d_model must be divisible by 2 * num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_head = d_head
        self.lambda_init = lambda_init

        # Learnable weights for Q, K, V
        self.WQ = nn.Parameter(torch.randn(d_model, 2 * d_head))
        self.WK = nn.Parameter(torch.randn(d_model, 2 * d_head))
        self.WV = nn.Parameter(torch.randn(d_model, 2 * d_head))

        # Learnable scalars for λ
        self.lambda_q1 = nn.Parameter(torch.randn(d_head))
        self.lambda_k1 = nn.Parameter(torch.randn(d_head))
        self.lambda_q2 = nn.Parameter(torch.randn(d_head))
        self.lambda_k2 = nn.Parameter(torch.randn(d_head))

    def forward(self, X):
        N, d_model = X.shape

        # Compute Q1, Q2, K1, K2, V
        Q = torch.matmul(X, self.WQ)
        K = torch.matmul(X, self.WK)
        V = torch.matmul(X, self.WV)
        Q1, Q2 = Q[:, :self.d_head], Q[:, self.d_head:]
        K1, K2 = K[:, :self.d_head], K[:, self.d_head:]

        # Compute λ
        lambda_val = (
            torch.exp(self.lambda_q1 * self.lambda_k1)
            - torch.exp(self.lambda_q2 * self.lambda_k2)
            + self.lambda_init
        )

        # Differential Attention
        attn1 = F.softmax(torch.matmul(Q1, K1.T) / (self.d_head ** 0.5), dim=-1)
        attn2 = F.softmax(torch.matmul(Q2, K2.T) / (self.d_head ** 0.5), dim=-1)
        diff_attn = attn1 - lambda_val * attn2

        # Weighted sum over values
        output = torch.matmul(diff_attn, V)
        return output


In [24]:
class MultiHeadDifferentialAttention(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(MultiHeadDifferentialAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.d_head = d_head

        # Create a DifferentialAttention module for each head
        self.heads = nn.ModuleList([
            DifferentialAttention(d_model, 1, d_head, lambda_init) for _ in range(num_heads)
        ])

        # Final linear layer to combine heads
        self.out_proj = nn.Linear(num_heads * d_head, d_model)

    def forward(self, X):
        head_outputs = [head(X) for head in self.heads]
        concatenated = torch.cat(head_outputs, dim=-1)  # Concatenate along feature dimension
        return self.out_proj(concatenated)


In [25]:
# Replace in a transformer block
class CustomTransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_head, lambda_init=0.8):
        super(CustomTransformerBlock, self).__init__()
        self.diff_attention = MultiHeadDifferentialAttention(d_model, num_heads, d_head, lambda_init)
        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model),
        )
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)

    def forward(self, X):
        attn_output = self.diff_attention(X)
        X = self.norm1(X + attn_output)
        ff_output = self.feed_forward(X)
        return self.norm2(X + ff_output)


In [26]:
from transformers.models.gpt2.modeling_gpt2 import GPT2Model, GPT2Block
from torch.nn import Module

class CustomGPT2Block(GPT2Block):
    def __init__(self, config, use_diff_attention=False, diff_attn_params=None):
        super().__init__(config)
        self.use_diff_attention = use_diff_attention

        if use_diff_attention:
            d_model = config.hidden_size
            num_heads = config.num_attention_heads
            d_head = d_model // num_heads

            # Initialize the differential attention
            self.attn = MultiHeadDifferentialAttention(
                d_model=d_model,
                num_heads=num_heads,
                d_head=d_head,
                lambda_init=diff_attn_params.get("lambda_init", 0.8),
            )

    def forward(self, hidden_states, layer_past=None, attention_mask=None, use_cache=False):
        if self.use_diff_attention:
            attn_output = self.attn(hidden_states)
        else:
            attn_output = super().attn(hidden_states, layer_past, attention_mask, use_cache)
        return super().mlp(attn_output)


# 3. Using your fine-tuned model, replace between 25% to 50% of the encoder and/or decoder layers’ multihead attention with multi-head differential attention. For each configuration you propose, train the new model, and assess the performance

In [28]:
from transformers import GPT2Config, GPT2LMHeadModel

def modify_gpt2_model(model, replace_fraction=0.25, lambda_init=0.8):
    """
    Modify the GPT-2 model by replacing a fraction of its attention layers with differential attention.
    """
    total_layers = len(model.transformer.h)
    num_layers_to_replace = int(total_layers * replace_fraction)
    layers_to_replace = sorted(range(total_layers))[-num_layers_to_replace:]  # Replace last layers

    for i in layers_to_replace:
        model.transformer.h[i] = CustomGPT2Block(
            config=model.config,
            use_diff_attention=True,
            diff_attn_params={"lambda_init": lambda_init},
        )
    return model

# Load the fine-tuned model and apply modifications
model_name = "./fine_tuned_aragpt2"  # Path to fine-tuned model
model = GPT2LMHeadModel.from_pretrained(model_name)
modified_model = modify_gpt2_model(model, replace_fraction=0.25)


In [30]:
from transformers import GPT2Config, GPT2LMHeadModel

def modify_gpt2_model(model, replace_fraction=0.50, lambda_init=0.8):
    """
    Modify the GPT-2 model by replacing a fraction of its attention layers with differential attention.
    """
    total_layers = len(model.transformer.h)
    num_layers_to_replace = int(total_layers * replace_fraction)
    layers_to_replace = sorted(range(total_layers))[-num_layers_to_replace:]  # Replace last layers

    for i in layers_to_replace:
        model.transformer.h[i] = CustomGPT2Block(
            config=model.config,
            use_diff_attention=True,
            diff_attn_params={"lambda_init": lambda_init},
        )
    return model

# Load the fine-tuned model and apply modifications
model_name = "./fine_tuned_aragpt2"  # Path to fine-tuned model
model = GPT2LMHeadModel.from_pretrained(model_name)
modified_model = modify_gpt2_model(model, replace_fraction=0.50)
